In [2]:
#네이버
import pandas as pd 

#해당 링크는 한국거래소에서 상장법인목록을 엑셀로 다운로드하는 링크입니다.
#다운로드와 동시에 Pandas에 excel 파일이 load가 되는 구조입니다.
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] 
#stock_code.head()

# 데이터에서 정렬이 따로 필요하지는 않지만 테스트겸 Pandas sort_values를 이용하여 정렬을 시도해봅니다.
stock_code.sort_values(['상장일'], ascending=True)

# 필요한 것은 "회사명"과 "종목코드" 이므로 필요없는 column들은 제외
stock_code = stock_code[['회사명', '종목코드']] 

# 한글 컬럼명을 영어로 변경 
stock_code = stock_code.rename(columns={'회사명': 'company', '종목코드': 'code'}) 
#stock_code.head()

# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
stock_code.code = stock_code.code.map('{:06d}'.format) 

company='LG화학' 
code = stock_code[stock_code.company==company].code.values[0].strip() ## strip() : 공백제거

df_naver = pd.DataFrame()
for page in range(1,123):
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code)     
    url = '{url}&page={page}'.format(url=url, page=page)
    df_naver = df_naver.append(pd.read_html(url, header=0)[0], ignore_index=True)
    

# df.dropna()를 이용해 결측값 있는 행 제거 
df_naver = df_naver.dropna() 

# 한글로 된 컬럼명을 영어로 바꿔줌 
df_naver = df_naver.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
# 데이터의 타입을 int형으로 바꿔줌 
df_naver[['close', 'diff', 'open', 'high', 'low', 'volume']] = df_naver[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 

# 컬럼명 'date'의 타입을 date로 바꿔줌 
df_naver['date'] = pd.to_datetime(df_naver['date']) 

# 일자(date)를 기준으로 오름차순 정렬 
df_naver = df_naver.sort_values(by=['date'], ascending=True) 

# 상위 5개 데이터 확인 
df_naver

,date,close,diff,open,high,low,volume
1828,2015-11-13,299500,1500,295000,305000,294000,228823
1827,2015-11-16,294000,5500,294000,298000,293000,140792
1826,2015-11-17,313500,19500,301000,316500,298500,336995
1825,2015-11-18,313000,500,313500,319000,310000,174571
1824,2015-11-19,326000,13000,317500,327500,317000,319695
...,...,...,...,...,...,...,...
5,2020-10-28,642000,10000,647000,647000,626000,368105
4,2020-10-29,651000,9000,629000,654000,629000,386081
3,2020-10-30,611000,40000,647000,648000,611000,820454
2,2020-11-02,623000,12000,609000,625000,588000,505856


In [ ]:
#야후

In [3]:
stockname = str(input())

LG화학


In [5]:
import pandas_datareader as pdr
# 회사명으로 주식 종목 코드를 획득할 수 있도록 하는 함수
def get_code(df, name):
    code = df.query("name=='{}'".format(name))['code'].to_string(index=False)
# 위와같이 code명을 가져오면 앞에 공백이 붙어있는 상황이 발생하여 앞뒤로 sript() 하여 공백 제거
    code = code.strip()
    return code
# excel 파일을 다운로드하는거와 동시에 pandas에 load하기
# 흔히 사용하는 df라는 변수는 data frame을 의미합니다.
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
# data frame정리
code_df = code_df[['회사명', '종목코드']]
# data frame title 변경 '회사명' = name, 종목코드 = 'code'
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})
# 종목코드는 6자리로 구분되기때문에 0을 채워 6자리로 변경
code_df.code = code_df.code.map('{:06d}'.format)
# ex) 삼성전자의의 코드를 구해보겠습니다.
code = get_code(code_df, stockname)
# yahoo의 주식 데이터 종목은 코스피는 .KS, 코스닥은 .KQ가 붙습니다.
# 삼성전자의 경우 코스피에 상장되어있기때문에 '종목코드.KS'로 처리하도록 한다.
code = code + '.KS'
# get_data_yahoo API를 통해서 yahho finance의 주식 종목 데이터를 가져온다.
df_yahoo = pdr.get_data_yahoo(code)

df_yahoo

ModuleNotFoundError: No module named 'pandas_datareader'